In [0]:
# 필수 라이브러리 설치
%pip install xgboost mlflow scikit-learn pandas --upgrade --prefer-binary

# 설치 후 커널 자동 재시작 (코드 실행 전 세션 초기화)
dbutils.library.restartPython()

#### 중복 생성 방지 및 재현율 개선 버전(2026-02-24)


In [0]:
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split

# --- [설정부: 유저 환경에 맞춰 수정] ---
CATALOG = "signalcraft_databricks"    # 실제 카탈로그명
SCHEMA = "default"  # 실제 스키마명
MODEL_NAME = f"{CATALOG}.{SCHEMA}.churn_predictor"

# UC 모드 활성화
mlflow.set_registry_uri("databricks-uc")

# --- [1단계: Spark 데이터 전처리 (안정성 및 성능 최적화)] ---

# 1. 테이블 로드 및 이력 데이터 사전 집계
snapshot_df = spark.table(f"{CATALOG}.{SCHEMA}.dlt_gold_user_behavior_snapshot")
history_df = spark.table(f"{CATALOG}.{SCHEMA}.dlt_silver_daily_watch_time_rt") \
    .select("user_id", "event_date").withColumnRenamed("event_date", "activity_date")

h_agg = history_df.select("user_id", "activity_date").distinct()

# 2. 시차 레이블링 Join (수정 포인트: 조인 키를 리스트로 전달)
# 이렇게 하면 결과 데이터프레임에 user_id가 딱 하나만 남습니다.
training_set_spark = snapshot_df.join(
    h_agg,
    (snapshot_df["user_id"] == h_agg["user_id"]) & 
    (h_agg["activity_date"] > snapshot_df["event_date"]) & 
    (h_agg["activity_date"] <= F.date_add(snapshot_df["event_date"], 7)),
    "left_outer"
).drop(h_agg["user_id"]) # 조인 후 h 테이블의 user_id를 명시적으로 제거

# 3. 레이블 생성 및 Safe Zone 필터링
max_history_date = history_df.select(F.max("activity_date")).collect()[0][0]

# snapshot_cols를 미리 정의 (중복 방지를 위해 select로 명시)
snapshot_cols = snapshot_df.columns

training_set_spark = training_set_spark.groupBy(snapshot_cols).agg(
    F.max("activity_date").alias("latest_activity")
).withColumn(
    "label", 
    F.when(F.col("latest_activity").isNull(), 1).otherwise(0)
).filter(
    F.col("event_date") <= F.date_sub(F.lit(max_history_date), 7)
).drop("latest_activity")

# 4. Pandas 변환 및 시계열 정렬 (Random Split 방지)
df = training_set_spark.toPandas()
df['event_date'] = pd.to_datetime(df['event_date'])
df = df.sort_values('event_date').reset_index(drop=True)

# --- [2단계: 피처 엔지니어링 및 인코딩] ---

# Ordinal Encoding: 위험도 순서 부여
risk_map = {'Active': 0, 'Soft Churn': 1, 'Dormant': 2, 'Churned': 3}
df['risk_level_encoded'] = df['churn_risk_level'].map(risk_map).fillna(0)

# One-Hot Encoding: 범주형 변수 처리
df = pd.get_dummies(df, columns=['segment'], prefix='seg')

# 핵심 신호: 변화율(Ratio) 피처 생성
df['watch_time_ratio'] = df['watch_time_7d_min'] / (df['watch_time_30d_min'] / 4 + 1)
df['active_days_ratio'] = df['active_days_7'] / (df['active_days_30'] / 4 + 1)

# 학습 피처 리스트 (데이터 누수 위험이 있는 churn_reason은 제외)
features = [
    'daily_watch_time_min', 'watch_time_7d_min', 'watch_time_30d_min',
    'active_days_7', 'active_days_30', 'days_since_last_login',
    'watch_time_ratio', 'active_days_ratio', 'risk_level_encoded'
]

X = df[features]
y = df['label']

# 클래스 분포 확인
class_counts = y.value_counts()
print(f"전체 클래스 분포:\n{class_counts}")

if len(class_counts) < 2:
    raise ValueError(f"학습 데이터에 클래스가 하나뿐입니다. (현재 클래스: {class_counts.index.tolist()}) "
                     "데이터 필터링 조건이나 기간을 확인하세요.")

# 시계열 분리 후 다시 확인
split_idx = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# 학습셋에 두 클래스가 모두 있는지 최종 확인
train_classes = y_train.unique()
if len(train_classes) < 2:
    print("⚠️ 경고: 학습셋에 클래스가 부족합니다. 무작위 분리로 일시 전환합니다.")
    # 시계열 순서보다 클래스 균형이 우선인 경우 (데이터가 적을 때)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# --- [3단계: 모델 학습 및 MLflow 등록] ---

# 재현율 강화를 위한 가중치 설정 (기존 비율의 1.8배)
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    scale_pos_weight=pos_weight,
    eval_metric='aucpr',           # Precision-Recall 최적화
    early_stopping_rounds=50,
    tree_method='hist',            # 서버리스 속도 최적화
    random_state=42
)

with mlflow.start_run(run_name="SignalCraft_Churn_Run_v4") as run:
    # 1. 모델 학습
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=10
    )

    # 2. 예측 및 확률값 계산
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred_05 = (y_proba > 0.5).astype(int)       # 기본 임계값 기준
    
    # 3. 메트릭 계산 및 로깅 (MLflow Dashboard에서 확인 가능)
    # 기본 임계값(0.5) 기준 지표
    mlflow.log_metric("auc_roc", roc_auc_score(y_test, y_proba))
    mlflow.log_metric("recall_0.5", recall_score(y_test, y_pred_05))
    mlflow.log_metric("precision_0.5", precision_score(y_test, y_pred_05))

    # 4. 모델 및 파라미터 저장
    mlflow.log_params(model.get_params()) # 하이퍼파라미터 자동 저장
    
    # 모델 등록 및 Signature 자동 생성
    signature = mlflow.models.infer_signature(X_train, model.predict(X_train))
    model_info = mlflow.xgboost.log_model(
        model, 
        artifact_path="model",
        signature=signature,
        input_example=X_train.head(3),
        registered_model_name=MODEL_NAME
    )
    
    # Alias 설정
    client = mlflow.tracking.MlflowClient()
    client.set_registered_model_alias(MODEL_NAME, "latest", model_info.registered_model_version)

print(f"✅ 모델 및 메트릭 저장 완료 (Version: {model_info.registered_model_version})")
